In [2]:
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import string

In [3]:
file = 'keywords_data.csv'
df_train = pd.read_csv(file)

y = df_train.Tag

In [4]:
# This is the a word tokenizer later user with the TfidfVectorizer
def word_tokenizer(text, stem=True):
    #tokenizes and stems the text
    
    tokens = word_tokenize(str(text).lower())
    punctuation = string.punctuation + '’'
    tokens = list(filter(lambda token: token not in punctuation, tokens))

    if stem==True:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
    else:
        tokens = [t for t in tokens if t not in stopwords.words('english')]

    return tokens

In [21]:
from gensim import corpora
from gensim.models.tfidfmodel import TfidfModel

In [31]:
# Remove common words and tokenize
# do it using the routine created before
corpus = [
    word_tokenizer(document, False)
    for document in df_train.Terms.to_list()
]

In [32]:
dictionary = corpora.Dictionary(corpus)

In [33]:
# The gensim TF-IDF model requires a data structure in a Bag of Word format (which is still a vector).
new_corpus = [dictionary.doc2bow(text) for text in corpus]
tfidf_model = TfidfModel(new_corpus, smartirs='Lpc')
corpus_tfidf = tfidf_model[new_corpus]

In [34]:
from gensim.matutils import corpus2csc

X = corpus2csc(corpus_tfidf).transpose()

In [35]:
X.shape

(16496, 3626)

In [38]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X.toarray(), y)

y_pred = clf.predict(X.toarray())

In [43]:
file = './Performance-2020-08-30.xlsx'
df = pd.read_excel(file, sheet_name=0)

In [44]:
unseen_tokens = [word_tokenizer(document, False) for document in df.Query.to_list()]
unseen_bow = [dictionary.doc2bow(t) for t in unseen_tokens]
unseen_vectors = tfidf_model[unseen_bow]
X = corpus2csc(unseen_vectors).transpose() # here I get the errors in the first screenshot

/Users/andreamoro/.pyenv/versions/3.8.5/lib/python3.8/site-packages/gensim/models/tfidfmodel.py:188: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Users/andreamoro/.pyenv/versions/3.8.5/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [42]:
X.shape

(1000, 3619)

In [40]:
y_pred = clf.predict(X.toarray())

/Users/andreamoro/.pyenv/versions/3.8.5/lib/python3.8/site-packages/gensim/models/tfidfmodel.py:188: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Users/andreamoro/.pyenv/versions/3.8.5/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3626 is different from 3619)